## l_______________________________________________________________________

### Feriados

Janeiro
01	TER	Ano-Novo
Abril
19	SEX	Sexta-feira Santa
21	DOM	Tiradentes
21	DOM	Páscoa

In [ ]:
hollydays=['2018-12-25', '2018-12-31','2019-01-01', '2019-03-04','2019-03-05','2019-03-06']
for i in ['01','19','21']:
    hollydays.append('2019-04-'+i)
#hollydays = [pd.Timestamp(i) for i in hollydays]

In [ ]:
hollydays

In [ ]:
def drop_dates(df, dates, inplace=False):
    """
        inputs:
            df -> a dataframe indexed by dates
            dates -> a list of str '%YYYY-MM-DD' dates
            inplace -> similar to pd.drop inplace parameter
        returns:
            df where dates are dropped if inplace is False
            None if inplace is True (similar to pd.drop)
    """
    to_drop_dates = list()
    for d in dates:
        to_drop_dates = to_drop_dates + list(df[d].index)
    dfo = df.drop(to_drop_dates,axis=0, inplace=inplace)
    return dfo

## l_______________________________________________________________________

In [ ]:
import pickle
processados = pickle.load(open('../pickles/all/processados.pkl','rb'))

In [ ]:
erros = pickle.load(open('../pickles/all/erros.pkl','rb'))

## l_______________________________________________________________________

In [ ]:
import json
import os
import pandas as pd
from genData import load_model

import json
import os
import pandas as pd
from genData import load_model
import re
import numpy as np
import pickle

from importlib import reload
import mainlib
reload(mainlib)
import A_base as ab
reload(ab)
from  mainlib import fasorial, lista, sdict, mm, mm_sep


import collections
import of_proc
reload(of_proc)
from of_proc import *

# Informações

In [ ]:
def read_info(path = '../info/info.json'):
    global PATH, OUTPUT_PATH, PROCESSED_PATH, PERFIS_PATH, HORARIOS_PATH, PATH_MM, PATH_FAS, \
    DEG, MOR, PER, FAS, OUTPUT_HEADER,FAS_CONCAT, MM_CONCAT, PICKLES_CONCAT, OUT_INDEX
    with open(path) as f:
        info = json.load(f)
        PATH = info['paths']['fasoriais']['input']
        OUTPUT_PATH = info['paths']['fasoriais']['output']
        PROCESSED_PATH = info['paths']['fasoriais']['processed']
        PERFIS_PATH = info['paths']['perfis']['perfis']
        HORARIOS_PATH = info['paths']['perfis']['horarios']
        PATH_MM = info['paths']['mm']['input']
        PATH_FAS = info['paths']['fasoriais']['input']
        DEG = info['paths']['nn_models']['degrau']
        MOR = info['paths']['nn_models']['morro']
        PER = info['paths']['nn_models']['perfil']
        FAS = info['paths']['nn_models']['fasorial']
        FAS_CONCAT =  info['paths']['concat']['fas']
        MM_CONCAT =  info['paths']['concat']['mm'] 
        PICKLES_CONCAT=  info['paths']['concat']['pickles']        
        OUTPUT_HEADER = info['output']['header']
        OUT_INDEX = info['output']['header']

In [ ]:
read_info()

In [ ]:
OUT_CONCAT = '../outputs/concat/'
OUT = '../outputs/all/'

In [ ]:
#preguiça -- arrumar
out_index = OUT_INDEX

### Carregar modelos neurais

# Coleções principais

#### Lista principal

In [ ]:
global LISTA_CAD

#### Modelos neurais

In [ ]:
global PERFIL, DEGRAU, MORRO, NN_FAS
PERFIL, DEGRAU, MORRO, NN_FAS = load_models(PER, DEG, MOR, FAS)

#### Arquivos

In [ ]:
global files_fas, files_mm

#### Perfis e horários

In [ ]:
global HORARIOS, PERFIS, ATIVS, DPERFIS, DPERFIS_NORMAL, DPERFIS_TROCA

#### Estruturas auxiliares

In [ ]:
global dim, dmi, df_fas, dfiles_mm, dfiles_fas_troca, dfiles_mm_troca, dfiles_fas_normal, dfiles_mm_normal
global o_i_normal, o_i_troca, o_instas, list_i_normal, list_i_troca, list_intas

## Lista de medidores e lista por horários

In [ ]:
LISTA_CAD = get_list()

In [ ]:
HORARIOS, PERFIS, ATIVS = get_horario_info()

### Dicionários e listas

In [ ]:
dim, dmi = dicts(LISTA_CAD)

In [ ]:
#gls i_normal -> instalações normal
# i_troca -> instalações com troca
#o_i_normal -> ordenado instalações normal
o_i_normal = collections.OrderedDict(sorted(LISTA_CAD.get_users(index = 'instalacao', troca=0,tipo='dictionary').items()))
o_i_troca = collections.OrderedDict(sorted(LISTA_CAD.get_users(index = 'instalacao', troca=1,tipo='dictionary').items()))
o_instas =  collections.OrderedDict(sorted({**o_i_normal, **o_i_troca}.items()))

##### Dicionário com os horário de funcionamento de cada cliente

In [ ]:
DPERFIS = sdict()
for i in PERFIS.index:
    j = PERFIS.loc[i,'Ramo de Atividade']
    if HORARIOS.index.contains(j):
        DPERFIS[i] = list(HORARIOS.loc[j,:])
    else:
        DPERFIS[i] = list(HORARIOS.iloc[0,:])

DPERFIS_NORMAL={}
for i in DPERFIS:
    if i in o_i_normal:
        DPERFIS_NORMAL[i] = o_i_normal[i]

DPERFIS_TROCA={}
for i in DPERFIS:
    if i in o_i_troca:
        DPERFIS_TROCA[i] = o_i_troca[i]

In [ ]:
dfiles_fas, nf_fas=get_files_3(list(o_i_normal.keys()), dim,  PATH_FAS)
dfiles_mm, nf_mm = get_files_3(list(o_i_normal.keys()),dim , PATH_MM)

In [ ]:
dfiles_fas2 = deslistar(dfiles_fas)
dfiles_mm2 = deslistar(dfiles_mm)

In [ ]:
dfiles_fas_concat, nf_fas_concat = get_files_2( list(o_i_troca.keys()), FAS_CONCAT)
dfiles_mm_concat, nf_mm_concat = get_files_2( list(o_i_troca.keys()), MM_CONCAT)

## l_______________________________________________________________________

## l_______________________________________________________________________

In [ ]:
dfiles_fas_concat = get_files_troca(dfiles_fas_concat)

In [ ]:
dfiles_mm_concat= get_files_troca(dfiles_mm_concat)

In [ ]:
odf_concat = collections.OrderedDict(sorted(dfiles_fas_concat.items()))

In [ ]:
odmm_concat = collections.OrderedDict(sorted(dfiles_mm_concat.items()))

## l_______________________________________________________________________

#### Nova função de corrente zero

In [ ]:
def new_corrent_zero(grid):
    pAmp = 20
    relacao = 400/5
    rdias = 0
    rminutos = 0
    severidade_regra = 0 #certeza de que nao tem nada errado
    cdata = -1
    
    corrente_referencia = pAmp/relacao
    rise = {}
    rise['IA'] = (grid['IA'] >= corrente_referencia/3).any()
    rise['IB'] = (grid['IB'] >= corrente_referencia/3).any()
    rise['IC'] = (grid['IC'] >= corrente_referencia/3).any()

#     print("Corrente de referencia:", corrente_referencia)
#     print("iA tem valores maiores que referencia/3: ", iA_levanta)
#     print("iB tem valores maiores que referencia/3: ", iB_levanta)
#     print("iC tem valores maiores que referencia/3: ", iC_levanta)

    for index, row in grid.iterrows():
        data = index.date()
        datastr = index.strftime('%Y-%m-%d')
        hora = index.time().hour
        correntes={}
        correntes['IA'] = float(row['IA'])*relacao
        correntes['IB'] = float(row['IB'])*relacao
        correntes['IC'] = float(row['IC'])*relacao
        zeros = {'IA':0, 'IB':0, 'IC':0}
        for i in ['IA', 'IB', 'IC']:
            if (cdata == -1 or cdata != data):
                #feriado = verificaFeriado(datastr, feriados)
                #print("data: ", datastr, feriado)
                if (cdata != -1):
                    ocorrencias = (horas*4)*0.7 #70%do tempo
                    #print(cdata,"Ref Ocorrencias:",ocorrencias,"IA",zeroiA,naozeroiA,"IB",zeroiB,naozeroiB,"IC",zeroiC,naozeroiC)
                    #print(zeros[i])
                    #continue
                    if (zeros[i] >= ocorrencias): # and naozeroiA == 0):
                        rdias +=1
                        rminutos +=(zeros[i]*15)                  
                #novo dia, começa a contar novamente
                cdata = data
                for i in correntes:
                    correntes[i]=0
                for i in zeros:
                    zeros[i] = 0

            horas= int(grid[datastr][i].resample('H').count().resample('D').count())
            #hf,horas = horarioFuncionamento(hora, dia, semanai, semanaf, sabadoi, sabadof, domingoi, domingof)
            #print(hf,cdata,hora,dia,iA,iB,iC)
            if (correntes[i] == 0):
                severidade_regra = 1
                if (correntes['IA']+correntes['IB']+correntes['IC'] >= pAmp):
                    #condição C = corrente zero e a soma das outras duas é maior ou igual a pAmp
                    zeros[i] +=1
                elif (not rise[i]):
                    #condição A = corrente zero na fase A sendo que a fase A nunca ultrapassa 1/3 de pAmp
                    zeros[i] +=1

    #so para ajustar severidade_regra
    if (rdias >= 10): # nao importa rminutos
        severidade_regra = 2
        
    return rdias, rminutos, severidade_regra

In [ ]:
#Nova implementação para contabilização de corrente zero
#Novidades:
# a) Lista de feriados
# b) Regra de contabilização nao se restringe à corrente ser zero apenas dentro do horário de funcionamento e do feriado

import datetime
from datetime import datetime
from datetime import time
from dateutil import parser

# def parseDate(strdate):
#     #format ddmmyyyyhhMMss
#     try:
#         result = parser.parse(strdate[0:2]+"/"+strdate[3:5]+"/"+strdate[6:10]+" "+strdate[11:13]+":"+strdate[14:16]+":00", dayfirst=True)
#     except ValueError:
#         #print (strdate)
#         result = parser.parse("01/01/1980 00:00:00", dayfirst=True)
#     return result
        
def horarioFuncionamento(hora, dia, semanai=8, semanaf=18, sabadoi=8, sabadof=18, domingoi=8, domingof=18):
    # veja que estou tratando String e tirei as acentuações do arquivo CSV
    # se voce tiver jeito melhor de fazer pode mudar
    if (dia == "Sabado"):
        horas = sabadof - sabadoi
        return (hora >= sabadoi and hora < sabadof), horas
    if (dia == "Domingo"):
        horas = domingof - domingoi
        return (hora >= domingoi and hora < domingof), horas
    else:
        horas = semanaf - semanai
        return (hora >= semanai and hora < semanaf), horas
    
def verificaFeriado(datastr, feriados):
    return (feriados['Feriados'] == datastr).any()
    
def NEWcorrenteZero(grid, feriados, pAmp, relacao, semanai=8, semanaf=18, sabadoi=8, sabadof=18, domingoi=8, domingof=18):
    rdias = 0
    rminutos = 0
    severidade_regra = 0 #certeza de que nao tem nada errado
    cdata = -1
    
    corrente_referencia = pAmp/relacao
    IA_levanta = (grid['IA'] >= corrente_referencia/3).any()
    IB_levanta = (grid['IB'] >= corrente_referencia/3).any()
    IC_levanta = (grid['IC'] >= corrente_referencia/3).any()
    za, zb, zc = [], [], []
    print("Corrente de referencia:", corrente_referencia)
    print("IA tem valores maiores que referencia/3: ", IA_levanta)
    print("IB tem valores maiores que referencia/3: ", IB_levanta)
    print("IC tem valores maiores que referencia/3: ", IC_levanta)
    for index, row in grid.iterrows():
        datetime = parseDate(row['data'])
        data = datetime.date()
        datastr = datetime.strftime("%d/%m/%Y")
        hora = datetime.time().hour
        dia = row['Dia']
        IA = float(row['IA'])*relacao
        IB = float(row['IB'])*relacao
        IC = float(row['IC'])*relacao
        naozeroIA, naozeroIB, naozeroIC = 0,0,0
        if (cdata == -1 or cdata != data):
            feriado = verificaFeriado(datastr, feriados)
            #print("data: ", datastr, feriado)
            if (cdata != -1):
                ocorrencias = (horas*4)*0.7 #70%do tempo
                print(cdata,"Ref Ocorrencias:",ocorrencias,"IA",zeroIA,"IB",zeroIB,"IC",zeroIC)
                za.append(zeroIA)
                zb.append(zeroIB)
                zc.append(zeroIC)
                if (zeroIA >= ocorrencias): # and naozeroIA == 0):
                    rdias +=1
                    rminutos +=(zeroIA*15)
                else:
                    if (zeroIB >= ocorrencias): # and naozeroIB == 0):
                        rdias +=1
                        rminutos +=(zeroIB*15)
                    else:
                        if (zeroIC >= ocorrencias): # and naozeroIC == 0):
                            rdias +=1
                            rminutos +=(zeroIC*15)
            #novo dia, começa a contar novamente
            cdata = data
            zeroIA, zeroIB, zeroIC = 0,0,0
            naozeroIA, naozeroIB, naozeroIC = 0,0,0
                   
        hf,horas = horarioFuncionamento(hora, dia, semanai, semanaf, sabadoi, sabadof, domingoi, domingof)
        #print(hf,cdata,hora,dia,IA,IB,IC)
        
        if (hf and not feriado):
            if (IA == 0):
                severidade_regra = 1
                if (IA+IB+IC >= pAmp):
                    #condição C = corrente zero e a soma das outras duas é maior ou igual a pAmp
                    zeroIA +=1
                else:
                    if (not IA_levanta):
                        #condição A = corrente zero na fase A sendo que a fase A nunca ultrapassa 1/3 de pAmp
                        zeroIA +=1
            else:
                naozeroIA +=1
            if (IB == 0):
                severidade_regra = 1
                if (IA+IB+IC >= pAmp):
                    #condição C = corrente zero e a soma das outras duas é maior ou igual a pAmp
                    zeroIB +=1
                else:
                    if (not IB_levanta):
                        #condição A = corrente zero na fase A sendo que a fase A nunca ultrapassa 1/3 de pAmp
                        zeroIB +=1
            else:
                naozeroIB +=1
            if (IC == 0):
                severidade_regra = 1
                if (IA+IB+IC >= pAmp):
                    #condição C = corrente zero e a soma das outras duas é maior ou igual a pAmp
                    zeroIC +=1
                else:
                    if (not IC_levanta):
                        #condição A = corrente zero na fase A sendo que a fase A nunca ultrapassa 1/3 de pAmp
                        zeroIC +=1
            else:
                naozeroIC +=1

    #so para ajustar severidade_regra
    if (rdias >= 10): # nao importa rminutos
        severidade_regra = 2
        
    return rdias, rminutos, severidade_regra

In [ ]:
#teste
file = 'from_w/BTI410315485-fasor_08552562_20190711_153409.csv'
grid=pd.read_csv(file,delimiter=';').astype('object')
file = 'from_w/feriados.csv'
feriados=pd.read_csv(file,delimiter=';').astype('object')
pAmp = 20
relacao = 400/5 # Observacao: cliente BT2A a relação é 1/1

#dias, minutos, severidade_regra = NEWcorrenteZero(grid, feriados, pAmp, relacao, 8, 18, 8, 18, 8, 18)
#print ("Dias com corrente zero em horario de funcionamento:",dias, " --- Minutos:", minutos, " (", severidade_regra, ")")

In [ ]:
def parseDate(strdate):
    #format ddmmyyyyhhMMss
    try:
        result = parser.parse(strdate[0:2]+"/"+strdate[3:5]+"/"+strdate[6:10]+" "+strdate[11:13]+":"+strdate[14:16]+":00", dayfirst=True)
    except ValueError:
        #print (strdate)
        result = parser.parse("01/01/1980 00:00:00", dayfirst=True)
    return result

## l_______________________________________________________________________

## Testes

## l-----------------------------------------------------------------------------------------------

## Corrente zero
##### Comparar resultado da nova função de corrente zero acima e a que fiz em pandas

## l-----------------------------------------------------------------------------------------------

In [ ]:
import mainlib
reload(mainlib)
from mainlib import fasorial, infos

hours = infos().hours
hollydays = infos().hollydays
ddays = infos().ddays



## Funções

In [ ]:
def proc_months(fas,less=False):
    for y in fas.months:
        for m in fas.months[y]:
            if less:
                fas.months[y][m].proc_fasorial_less()
            else:
                fas.months[y][m].proc_fasorial()

In [ ]:
def build_fas(ex1, proc=True, **kwargs):
    if ex1 in dfiles_fas_concat.keys():
        print('troca')
        file1 = dfiles_fas_concat[ex1]
        ddays=o_i_troca[ex1]
        path = FAS_CONCAT
        f1 = fasorial(name=ex1)
        f1.read_fasorial(os.path.join(path,file1), dtype=None)
    elif ex1 in dfiles_fas.keys():
        print('normal')
        file1 = dfiles_fas[ex1] 
        ddays=o_i_normal[ex1]
        path = PATH_FAS       
        f1 = fasorial(name=ex1)
        f1.read_fasorial(os.path.join(path,file1[0]), dtype=None)
    
    
    f1.split_months(ddays=ddays)
    if proc:
        proc_months(f1, **kwargs)
    return f1

In [ ]:
def proc2m(f1, proc=False):
    ld = sdict()
    for y in f1.months:
        for m in f1.months[y]:
            month = f1.months[y][m].df.copy()
            if proc:
                month = proc1m(month)
            ld[m] = month
    return ld

In [ ]:
def proc1m(m):
    m.reset_index(inplace=True, drop=False)
    m['Dia']= m['data'].apply(lambda x: x.weekday()).map(said)
    m['data'] =pd.to_datetime(m['data'])
    m['data']= m['data'].apply(lambda x: x.strftime('%d/%m/%Y %H:%M'))
    return m

## Variáveis

In [ ]:
dias = {
    "Segunda":0,
    "Terca":1,
    "Quarta":2,
    "Quinta":3,
    "Sexta":4,
    "Sabado":5,
    "Domingo":6
    
}
said = {v:k for k,v in dias.items()}

In [ ]:
hours = ['8:00', '18:00', '8:00', '18:00', '8:00', '18:00', '0:00', '0:00']

In [ ]:
#0-> segunda
#6 -> domingo
#7 feriado
hours2 = {
    0:('8:00', '18:00'),
    1:('8:00', '18:00'),
    2:('8:00', '18:00'),
    3:('8:00', '18:00'),
    4:('8:00', '18:00'),
    5:('8:00', '18:00'),
    6:('8:00', '18:00'),
    7:('0:00', '0:00'),
}

In [ ]:
hollydays = [
'2019-03-02',
'2019-03-03',
 '2019-03-04',
 '2019-03-05',
 '2019-03-06']

In [ ]:
# Criar um dicionário único para todos os caminhos de arquivos
ddirs={}
for u,v in dfiles_fas2.items():
    ddirs[u] = os.path.join(PATH_FAS,v)
for u,v in dfiles_fas_concat.items():
    if u in ddirs:
        print('Warning. Instalacao repetida')
    ddirs[u] = os.path.join(FAS_CONCAT, v)

In [ ]:
def get_fasorial(path,insta='not_found',iddays=default_ddays(),**kwargs):
    f = fasorial(name=insta, ddays = iddays)
    f.read_fasorial(path,**kwargs)
    return f



##  Criar um fasorial simples.

In [ ]:
import mainlib
reload(mainlib)
from mainlib import fasorial, mm

In [ ]:
ex1='BTI413195196'
i4 = 'BTI430044836'
f = get_fasorial(ddirs[i4], insta=i4, dtype=None)
df = f.df['2019-02']

#### check_fas

## l_______________________________________________________________________

In [ ]:
df,i = fasorial().check_df(pd.DataFrame({1:{2:3,4:5}}))

In [ ]:
df[0]

In [ ]:
df,i = f.check_df()
df

#### drop_fas

## l_______________________________________________________________________

In [ ]:
f.df.columns

In [ ]:
f.df.shape

In [ ]:
f.drop_fas()

In [ ]:
f.df.shape

In [ ]:
f.df.columns

#### string_types

In [ ]:
x=f.string_types()

In [ ]:
x

#### count_empty

In [ ]:
x = f.count_empty()

#### proc_string

In [ ]:
i4 = 'BTI430044836'
f = get_fasorial(ddirs[i4], insta=i4, dtype=None)
df = f.df['2019-02']

In [ ]:
df2=f.df.copy()

In [ ]:
df2=df2.applymap(f.proc_string)

In [ ]:
f.df['PB'].value_counts()

In [ ]:
df2['PB'].value_counts()

#### remove_dup

In [ ]:
i4 = 'BTI430044836'
f = get_fasorial(ddirs[i4], insta=i4, dtype=None)
df = f.df['2019-02']

In [ ]:
f.df.index.shape

In [ ]:
f.df.index.nunique()

In [ ]:
f.df.index.duplicated(keep='first').sum()

In [ ]:
f.remove_dup()

In [ ]:
f.shape_dup

In [ ]:
assert(f.shape_dup[0] == f.df.shape[0] + f.duplicates)

In [ ]:
f.duplicates

## l_______________________________________________________________________



####  BTI413195196

In [ ]:
ex1='BTI413195196'

In [ ]:
f1 = build_fas(ex1)
months = proc2m(f1)
months_proc = proc2m(f1, proc=True)

## l_______________________________________________________________________

## l_______________________________________________________________________

### Anotações

-999 na mm --> 'BTI410630442'

## l_______________________________________________________________________

### Checagem de métodos

In [ ]:
import copy

In [ ]:
exemplo = 'BT2A400029491'
arquivo = dfiles_fas[exemplo][0]
f = fasorial()
f.read_fasorial(os.path.join(PATH_FAS, arquivo), sep=';', dtype= None)

#### Overview

In [ ]:
f.df.describe()

In [ ]:
f.df.shape

In [ ]:
f.df.info()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(np.arange(len(f.df)),f.df.index)

### 1- fasorial.remove_dup()

In [ ]:
def check_dup(df):
    c=0
    c1=0
    for i in df.index:
        if c == i :
            c1 +=1
        c=i
    return c1
    

In [ ]:
def check_dup_2(df):
    return df.index.duplicated(keep='first').sum()

In [ ]:
print(check_dup(f.df), check_dup_2(f.df))

In [ ]:
print( len(f.df) - len(f.remove_dup(df = f.df.copy())))

In [ ]:
f.remove_dup()

## l_______________________________________________________________________

### 2-  fasorial.drop_hollydays()

# ????

## l_______________________________________________________________________

### 3-  fasorial.split_months
##### Pega 30 dias antes da data de leitura, exclusive a última

In [ ]:
months = {'Janeiro':1, 'Fevereiro':2, 'Março':3, 'Abril':4, 'Dezembro':12}
months2 = dict([[v,k] for k,v in months.items()])

In [ ]:
ddays = default_ddays()
ddays

In [ ]:
print(
    len(f.df[ddays[12] - pd.Timedelta(days=30):ddays[12]]),
    len(f.df[ddays[1] - pd.Timedelta(days=30):ddays[1]]),
    len(f.df[ddays[2] - pd.Timedelta(days=30):ddays[2]]),
    len(f.df[ddays[3] - pd.Timedelta(days=30):ddays[3]]),
    len(f.df[ddays[4] - pd.Timedelta(days=30):ddays[4]])
)

In [ ]:
f.split_months(ddays=ddays)
for y in f.months:
    for m in f.months[y]:
        print(len(f.months[y][m].df))

In [ ]:
for y in f.months:
    for m in f.months[y]:
        print(pd.Timestamp(f.months[y][m].df.index[0]).weekday())

In [ ]:
for y in f.months:
    for m in f.months[y]:
        print(months2[m])
        print(f.months[y][m].df.resample('D').count().resample('M').count().iloc[:,0])

## l_______________________________________________________________________

### 3-  fasorial.proc_fasorial()

In [ ]:
import mainlib

reload(mainlib)

from mainlib import fasorial

#provisório
exemplo = 'BT2A400029491'
arquivo = dfiles_fas[exemplo][0]
f = fasorial()
f.read_fasorial(os.path.join(PATH_FAS, arquivo), sep=';', dtype= None)
f.remove_dup()
f.split_months(ddays = default_ddays())

In [ ]:
jan = copy.deepcopy(f.months[2019][1])

daterange(jan.df)

In [ ]:
30*24*4

In [ ]:
jan.df.shape

In [ ]:
jan.df.info()

In [ ]:
jan.proc_fasorial()

In [ ]:
jan.df.shape

In [ ]:
assert jan.df.shape[0] == 30*24*4

In [ ]:
jan.outputs

## l_______________________________________________________________________

### 3-  fasorial.feat_eng_fasorial()

In [ ]:
jan.feat_eng_fasorial()

In [ ]:
jan.feat_eng_fasorial??

#### tension_angles

In [ ]:
jan.tension_angles??

In [ ]:
jan.df[['AVA','AVB', 'aab']]

In [ ]:
jan.df[['AVA','AVC', 'aac']]

In [ ]:
jan.df[['AVB','AVC', 'abc']]

#### min_of_three

In [ ]:
jan.feat_eng_fasorial??

In [ ]:
jan.df[['VA', 'VB', 'VC', 'minV']]

In [ ]:
jan.df[['IA', 'IB', 'IC','minA']]

In [ ]:
jan.df[['PA', 'PB', 'PC','minP']]

#### angle_check

In [ ]:
jan.angle_check??

In [ ]:
jan.df[['aab','abc','aac','angcheck']]

In [ ]:
jan.df[['aab','abc','aac','angcheck']][jan.df['angcheck']==True]

#### call_pot

In [ ]:
jan.call_pot??

In [ ]:
jan.df.columns

In [ ]:
jan.potNegativa??

### Summarize
##### <li> day_check <li> quantity_check <li> day_check_bool <li> quantity_check_bool <li> ult_corrente_zero <li> sunlight <li> cut_hours

In [ ]:
df = copy.deepcopy(jan.df)

##### day_check
##### <li> Dúvida. Atualmente ativa com apenas uma leitura no dia. Isso está certo?

In [ ]:
jan.day_check??

In [ ]:
df[df['minV']<0.0001]['minV'].resample('D').min().count()

##### quantity_check

In [ ]:
jan.quantity_check??

In [ ]:
jan.quantity_check('minV',threshold=0.0001)

In [ ]:
df[df['minV']<0.0001]['minV'].count()

day_check_bool

In [ ]:
jan.day_check_bool??

In [ ]:
jan.df.iloc[-600:]['angcheck'] = True

In [ ]:
jan.day_check_bool(jan.df.iloc[-600:])

In [ ]:
daterange(jan.df.iloc[-600:])

quantity_check_bool

In [ ]:
jan.quantity_check_bool??

In [ ]:
df[['aab','abc','aac']]

In [ ]:
df['angcheck']

cut_hours

In [ ]:
jan.cut_hours??

In [ ]:
hours

In [ ]:
hours2 = ['6:00', '7:00', '6:00', '7:00', '6:00', '7:00', '6:00', '7:00']

In [ ]:
daterange(jan.df)

In [ ]:
jan.cut_hours(hours2)

sun_light

In [ ]:
jan.sunlight??

In [ ]:
jan.sunlight()['2019-01-01']